In [11]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean, std
from sklearn.model_selection import KFold
import operator
import pickle
from joblib import dump, load

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score,precision_score,roc_auc_score,recall_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier


# # import nltk
# # from nltk.corpus import stopwords
# # from nltk.stem import PorterStemmer
# # from textblob import TextBlob, Word
# nltk.download('stopwords')
# nltk.download('punkt')

# stopwords = stopwords.words('english')
# # from nltk.stem import WordNetLemmatizer

from db_wrapper import db

In [273]:
import sys
sys.executable

'c:\\program files\\python35\\python.exe'

In [346]:
tfidf_vect = load(r"G:\feature.joblib") # Loading model joblib
model=load(r"G:\Reclassification_Algorithm\model.joblib") # Loading tfidf_vectorizer joblib
vocab_dict=load(r"G:\Reclassification_Algorithm\vocab.joblib")

In [275]:
select_query = "Select * from src.sap_ghq_reclass_data"

stat,result_set = db.select(select_query)
test_data = pd.DataFrame(result_set)

Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s Select * from src.sap_ghq_reclass_data 1
>>>> Executed Query -- %s with dbhndl id=%s Select * from src.sap_ghq_reclass_data 1


In [276]:
test_data.rename({"lifnr":"vendor id"}, axis=1, inplace=True)

In [277]:
test_data["vendor id"].unique()

array(['0000381111', '0000000276', '0000000525', ..., '0008349661',
       '0000320931', '0003008976'], dtype=object)

In [278]:
test_data.package.unique()

array(['Sales', None, 'Non ZBB - VLC Trans', 'People', 'Leases / rentals',
       'Services', 'Technology', 'Non ZBB', 'Legal', 'Maintenance',
       'Fees', 'Marketing', 'Utilities', 'Travel', 'Institutional',
       'Non ZBB - VLC LS', 'Supply Chain Overhead', 'Taxes',
       'Variable compensation'], dtype=object)

In [279]:
select_query = "Select * from [src].[sap_ghq_vendor_base]"

stat,result_set = db.select(select_query)
vendor_data = pd.DataFrame(result_set)

Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s Select * from [src].[sap_ghq_vendor_base] 1
>>>> Executed Query -- %s with dbhndl id=%s Select * from [src].[sap_ghq_vendor_base] 1


In [280]:
vendor_data.columns

Index(['(INV) CAPEX/OPEX', '(INV) Document Number (InvoiceId)',
       '(INV)Company Country (Company Country)',
       '(INV)Company Zone (Company Zone)',
       '(INV)Contract Id + Line item (Contract Id + Line item)',
       '(INV)Contract Id + Line item (Contract vs# Non-Contract)',
       '(INV)G/L Account (ZBB Package Description  )',
       '(INV)G/L Account (ZBB Subpackage Description  )',
       '(INV)Head office Supplier Name (Head office Supplier code)',
       '(INV)Head office Supplier Name (Head office Supplier name)',
       '(INV)Material Group (Material Group VBS description)',
       '(INV)Supplier (ERP Supplier)',
       '(INV)Supplier (Supplier Code (supplier# ID))',
       '(INV)Supplier (Supplier Global Ultimate Parent (enr))',
       '(INV)VBS Supplier description (VBS L1 name)',
       '(INV)VBS Supplier description (VBS L2 code)',
       '(INV)VBS Supplier description (VBS L2 name)',
       '(PO) Purchasing Organisation', '(PO)Company Site  (Site Id)',
       '

In [281]:
# vendor_data=pd.read_csv(r"G:\Users\C888653\Reclassification_Algorithm\\vendor base.csv")
vendor_data2=vendor_data[['(INV)Supplier (Supplier Code (supplier# ID))','(INV)G/L Account (ZBB Package Description  )',
       '(INV)G/L Account (ZBB Subpackage Description  )']]

In [282]:
vendor_data2.rename({'(INV)Supplier (Supplier Code (supplier# ID))' : "vendor id",
                     
                     '(INV)G/L Account (ZBB Package Description  )' : "package", 
                    
                    '(INV)G/L Account (ZBB Subpackage Description  )': "subpackage"}, inplace=True, axis=1)

In [283]:
def add0(x):
    try:
        if(len(x)!=10):
            padding_size=(10-len(x))
            x=x.rjust(10, "0")
            
        else:
            x
    except:
        print("error in ",x)
        
    return(float(x))
        

In [284]:
test_data["vendor id"]=test_data["vendor id"].astype(str)
test_data["vendor id"]=test_data["vendor id"].apply(lambda x: add0(x))

In [285]:
test_data["vendor id"]=test_data["vendor id"].round(decimals=0).astype(object)

In [286]:
vendor_data2["vendor id"]=vendor_data2["vendor id"].astype(str)
vendor_data2["vendor id"]=vendor_data2["vendor id"].apply(lambda x: add0(x))
vendor_data2["vendor id"]=vendor_data2["vendor id"].round(decimals=0).astype(object)

In [287]:
test_data["vendor id"][:3]


0    381111
1       276
2       525
Name: vendor id, dtype: object

In [288]:
vendor_data2["vendor id"][:3]

0    346426
1    383062
2    382548
Name: vendor id, dtype: object

In [ ]:
def punc_reomval(x):
    
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in x:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [ ]:
def preprocessing(x):
  
    x = re.sub(r'\W', ' ', str(x))
    x = re.sub(r'\s+[a-zA-Z]\s+', ' ', x)
    x = re.sub(r'\^[a-zA-Z]\s+', ' ', x) 
    x = re.sub(r'\s+', ' ', x, flags=re.I)
    return x

In [488]:
def appostrophe_replace(x):
    return(x.replace("'", " ").replace('"', " "))

In [489]:
test_data["purchase_order_description"]=test_data["purchase_order_description"].apply(lambda x: punc_reomval(x))
test_data["purchase_order_description"]=test_data["purchase_order_description"].apply(lambda x: preprocessing(x))
test_data["purchase_order_description"]=test_data["purchase_order_description"].apply(lambda x: appostrophe_replace(x))

In [491]:
# test_data[test_data["purchase_order_description"].str.contains("MXP")]

In [492]:
import ven_map
from ven_map import O2O, result

In [493]:
# data=pd.read_excel(r'G:\Users\C888653\Reclassification_Algorithm\ghq_data_with Vendor id.xlsx')

In [494]:
# test_data[test_data["package"]=="Technology"]["vendor id"].unique()

In [495]:
# vendor_data2["vendor id"].unique()

In [496]:
# tech=test_data[test_data.package=="Technology"]

In [497]:
# tech=test_data[test_data.package=="Technology"]

In [498]:
# tech["vendor id"]=tech["vendor id"].astype(str)
# tech["vendor id"]=tech["vendor id"].apply(lambda x: add0(x))
# tech["vendor id"]=tech["vendor id"].round(decimals=0).astype(object)

In [499]:
# tech["vendor id"].unique()

In [500]:
oTo_o=O2O(test_data)

In [501]:
oTo_o.count_pack_venid()[:3]

vendor id
1.0    1
Name: package, dtype: int64

In [502]:
oTo_o.count_subp_venid()[:3]

vendor id
1.0    1
Name: subpackage, dtype: int64

In [503]:
oTo_o.uniq_p

array(['Sales', None, 'Non ZBB - VLC Trans', 'People', 'Leases / rentals',
       'Services', 'Technology', 'Non ZBB', 'Legal', 'Maintenance',
       'Fees', 'Marketing', 'Utilities', 'Travel', 'Institutional',
       'Non ZBB - VLC LS', 'Supply Chain Overhead', 'Taxes',
       'Variable compensation'], dtype=object)

In [504]:
oTo_o.uniq_subp

array(['Commercial Agreement - Distributors', None, 'Other',
       'International Mobility', 'Temps Non-Payroll',
       'Lease of Real Estate', 'Cleaning and Gardening',
       'Digital Security', 'Non ZBB Accounts', 'Legal Fees',
       'Individual Benefits',
       'Industrial Equip & Machinery Maint. Materials (P&M)',
       'Consulting (project-based)', 'Application Support',
       'Agency Fees & Expenses', 'Office Supplies',
       'Document & Archiving Services', 'Internal Events Travel',
       'Internal Communication', 'International other',
       'Trade Marketing Material: Non Market Programs - Distributors',
       'Administrative Support', 'Outside Counsel Fees', 'Datacenter',
       'Consulting (recurring)', 'Laboratory Equip Repair & Maint.',
       'Corporate Affairs fees', 'Market Research - Working Money',
       'Brand Promotion', 'Production Supplies', 'Laboratory Services',
       'Building & Facilities Maint. Materials', 'Internal Events',
       'Sponsorship', 

In [505]:
len(oTo_o.uniq_venid)

1901

In [506]:
r=result(vendor_data2,tech)
df=r.result_O2O() 

In [507]:
df["reassign_pack"].value_counts()

Unassigned    721
Technology    301
OUT OF ZBB     13
Name: reassign_pack, dtype: int64

In [508]:
df["reassign_subpack"].value_counts()

Unassigned              837
Applications Support     63
Application Licenses     35
Security                 34
Workplace                34
Telecom Services         19
OUT OF ZBB               13
Name: reassign_subpack, dtype: int64

In [509]:
df.shape

(1035, 18)

In [510]:
o2o=df[~((df["reassign_pack"]=="Unassigned" ) | (df["reassign_pack"]=="OUT OF ZBB") | (df["reassign_subpack"]=="Unassigned" ))]

In [511]:
o2o.shape

(185, 18)

In [512]:
o2o["reassign_pack"].value_counts()

Technology    185
Name: reassign_pack, dtype: int64

In [513]:
o2m=df[((df["reassign_subpack"]=="Unassigned" ) | (df["reassign_subpack"]=="OUT OF ZBB") | (df["reassign_pack"]=="Unassigned" ) | (df["reassign_pack"]=="OUT OF ZBB"))]

In [514]:
o2m.shape[0]+o2o.shape[0]

1035

In [515]:
ip_data = pd.read_excel(r'G:\Users\C888653\Reclassification_Algorithm\Trainning dataset.xlsx')

In [516]:
testData=o2m[~o2m["po_key"].isin(ip_data["po_key"])]

In [517]:
testData.shape

(850, 18)

In [518]:
testData.isna().sum()

company_code                                0
description                                29
gl_code                                     0
gl_code_desc                                0
gpo categories                             29
vendor id                                   0
material_group_description                  0
package                                     0
po_key                                      0
po_total_amount_ordered_in_usd_currency     2
purchase_order_description                  0
sakto                                       0
subpackage                                  0
vendor_name                                 0
p_Match                                     0
sub_p_Match                                 0
reassign_pack                               0
reassign_subpack                            0
dtype: int64

In [519]:
# string="MXP Aug'19"
# string.replace("'", ' ')

In [520]:
from scipy.sparse import csr_matrix, hstack
def add_feature(X, feature_to_add):
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [521]:
# testData["purchase_order_description"]=testData["purchase_order_description"].apply(lambda x: punc_reomval(x))
# testData["purchase_order_description"]=testData["purchase_order_description"].apply(lambda x: preprocessing(x))
# testData["purchase_order_description"]=testData["purchase_order_description"].apply(lambda x: appostrophe_replace(x))

In [522]:
un_tf_idf_vect=tfidf_vect.transform(testData["purchase_order_description"].values)

In [523]:
un_tf_idf_vect

<850x3150 sparse matrix of type '<class 'numpy.float64'>'
	with 4303 stored elements in Compressed Sparse Row format>

In [524]:
test_vectorized = add_feature(un_tf_idf_vect, pd.Series(testData["purchase_order_description"].values).str.len())

In [525]:
test_vectorized

<850x3151 sparse matrix of type '<class 'numpy.float64'>'
	with 5153 stored elements in Compressed Sparse Row format>

In [526]:
test_df=pd.DataFrame(test_vectorized.todense(), index=testData["purchase_order_description"].values, columns=list(vocab_dict.values()))

In [527]:
test_df.shape

(850, 3151)

In [528]:
predicted=np.array(model.predict(test_df))

In [529]:
test_df["predicted"]=predicted

In [530]:
test_df["predicted"]

SSF Code Review                                Digital Security
CASB - NAZ                                     Digital Security
Pentest - People MDM                        Application Support
Gender Equality Comms                       Application Support
PO Extension of GCC Tickets/SRs for GOP     Application Support
SOW-MOUTS-2019-008 PMO Service              Application Support
CMS Support Azure h2 2019                            Datacenter
Azure Microsoft Q1                                   Datacenter
2019 IT2 Subscription and Maintenance       Application Support
Pentest - OPR Meeting                       Application Support
Azure costs for Q1 - 2019 OPEX              Application Support
Gain                                        Application Support
SHARP - CRQ00CHG0378653 - TSC Improv.       Application Support
SR PMDM Accenture                           Application Support
Insight Power BI licenses                   Application Support
SHARP - CRQ00CHG0378653 - Creat Fixes   

In [531]:
testData.drop(['p_Match', 'sub_p_Match', 'reassign_pack', 'reassign_subpack'], axis=1, inplace=True)

In [532]:
testData["predicted"]=predicted

In [533]:
testData["predicted"].value_counts()

Application Support     579
Workplace                90
Digital Security         76
Datacenter               74
Application Licenses     16
Telecom Services         13
Mobile Services           2
Name: predicted, dtype: int64

[po_key, po_decription, predicted, vendor logic flag]

In [534]:
o2o.drop(['p_Match', 'sub_p_Match', 'reassign_pack'], axis=1, inplace=True)

In [535]:
o2o.rename({"reassign_subpack": "predicted"}, axis=1, inplace=True)

In [536]:
o2o["flag"]=np.ones(o2o.shape[0])

In [537]:
testData["flag"]=np.zeros(testData.shape[0])

In [538]:
final_res1=o2o[[ 'po_key', 
       'purchase_order_description', 
       'predicted', 'flag']]

In [539]:
final_res2=testData[[ 'po_key', 
       'purchase_order_description', 
       'predicted', 'flag']]

In [540]:
final_res=pd.concat([final_res1, final_res2], axis=0)

In [541]:
final_res.reset_index(drop=True, inplace=True)

In [542]:
final_res.shape

(1035, 4)

In [557]:


final_res["purchase_order_description"]=final_res["purchase_order_description"].apply(lambda x: punc_reomval(x))
final_res["purchase_order_description"]=final_res["purchase_order_description"].apply(lambda x: preprocessing(x))
final_res["purchase_order_description"]=final_res["purchase_order_description"].apply(lambda x: appostrophe_replace(x))

In [560]:
i=0

for row in final_res.itertuples():
#     print(row[1],row[2])
    print(i)
    insert_query = "INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('%s','%s','%s','%s')"%(row[1],row[2], row[3], row[4])
    db.execute(insert_query)
    
    i=i+1
    
    

0
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450484629200010','Blackline2019','Applications Support','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450484629200010','Blackline2019','Applications Support','1.0') 1
1
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450477289200080','Account number562508364089 October','Applications Support','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450477289200080','Account number562508364089 October','Applications Support','1.0') 1
2
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450471029400010','B2B Support 2019','Applications Support','1.0') 1
>

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450531543700010','MuleSoft Sumologic Implementation','Applications Support','1.0') 1
22
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450483016500010','Stefanini Workplace JAG 2019','Workplace','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450483016500010','Stefanini Workplace JAG 2019','Workplace','1.0') 1
23
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450507988500020','Access Management Extension for 2019','Application Licenses','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450507988500020','Access Management Extension for 2019','Application Licenses','1.0') 1
24


>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450479947000020','post telecom subscr Feb','Telecom Services','1.0') 1
46
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450482513900010','AWS Hosting B2B','Applications Support','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450482513900010','AWS Hosting B2B','Applications Support','1.0') 1
47
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450477289200050','Account number562508364089 Jul','Applications Support','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450477289200050','Account number562508364089 Jul','Applications Support','1.0') 1
48
Found idle dbhandle id = %s 1


<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450472947500010','Pinpoint Support 2019','Applications Support','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450472947500010','Pinpoint Support 2019','Applications Support','1.0') 1
70
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450528093400010','Infosys GLASS changes','Applications Support','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450528093400010','Infosys GLASS changes','Applications Support','1.0') 1
71
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450479947000050','post telecom subscr May','Telecom Services','1.0') 1
>>>> Executed Query -- %s with dbhndl 

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450529693400010','TAKT Suporte','Applications Support','1.0') 1
91
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450473664700010','2019 Pitchbook subscription','Application Licenses','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450473664700010','2019 Pitchbook subscription','Application Licenses','1.0') 1
92
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450521905700010','OneTrust Renewal','Security','1.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450521905700010','OneTrust Renewal','Security','1.0') 1
93
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbh

...

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('480047457500010','Netbond 2019','Application Support','0.0') 1
937
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450514254200040','Access Mgmt Oct19','Application Support','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450514254200040','Access Mgmt Oct19','Application Support','0.0') 1
938
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450495433500010','CASB Renewal for GHQ','Digital Security','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450495433500010','CASB Renewal for GHQ','Digital Security','0.0') 1
939
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450472666100030','Gain','Application Support','0.0') 1
963
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450505470200030','Beedirect Injector','Workplace','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450505470200030','Beedirect Injector','Workplace','0.0') 1
964
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450518268800010','MLAN IS Companion Services','Telecom Services','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450518268800010','MLAN IS Companion Services','Telecom Services','0.0') 1
965
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s 

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450532130900010','SHARP CRQ00CHG0439697 Fix AP job','Application Support','0.0') 1
986
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450513231500010','CognosTM1 Accenture SRs 2019','Application Support','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450513231500010','CognosTM1 Accenture SRs 2019','Application Support','0.0') 1
987
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450529688900010','CRQ00CHG0434841 LAN Problem mgmt','Application Support','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450529688900010','CRQ00CHG0434841 LAN Problem mgmt','Application Support','0

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450508918600040','IT2 project IaaS DB support GHQ','Datacenter','0.0') 1
1008
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450503985100020','GLS Red','Application Support','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450503985100020','GLS Red','Application Support','0.0') 1
1009
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450508918600030','IT2 project IaaS DB support GHQ','Datacenter','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450508918600030','IT2 project IaaS DB support GHQ','Datacenter','0.0') 1
1010
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s w

>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('480047500300010','HOPPY Hard Coding and Speed 1191','Application Support','0.0') 1
1032
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('480046842500030','201819 Zaius Global Contract','Application Support','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('480046842500030','201819 Zaius Global Contract','Application Support','0.0') 1
1033
Found idle dbhandle id = %s 1
<<<< Executing Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450473026500050','Lego Block Support','Datacenter','0.0') 1
>>>> Executed Query -- %s with dbhndl id=%s INSERT INTO Spend.ghq_subpackage_reclassified_results VALUES('450473026500050','Lego Block Support','Datacenter','0.0') 1
1034
Found idle dbhandle id = %s 1
<<

#### 53, 93,114,174,177,